# Chat UI with API Test

This notebook demonstrates how to set up the Chat UI widget with its API functionality enabled.

In [ ]:
from chat_ui import ChatWidget, get_api_handler
import pandas as pd
import numpy as np
import time

# Create widget instance
chat = ChatWidget()

# Set up API
try:
    APIHandler = get_api_handler()
    api = APIHandler(
        chat_widget=chat,
        host='0.0.0.0',  # Important for Docker - allow external connections
        port=5000,
        api_key='test-key'
    )
    api.start(background=True)
    print(f"API available at {api.base_url}")
    print(f"API Key: test-key")
except ImportError as e:
    print(f"API not available: {e}")

# Create a sample artifact to test with
sample_df = pd.DataFrame({
    'Date': pd.date_range(start='2023-01-01', periods=5, freq='D'),
    'Value': np.random.randint(100, 1000, 5),
    'Category': ['A', 'B', 'A', 'C', 'B']
})

chat.create_artifact(
    "sample_dataframe",
    sample_df,
    "",
    "Sample DataFrame Artifact",
    "dataframe"
)

# Display widget
chat

## API Testing Instructions

You can test the API using curl commands from your host machine or another container. Here are example commands to test each endpoint:

In [ ]:
# Instructions for testing API
curl_commands = """
# Send a message via API
curl -X POST http://localhost:5000/api/v1/messages \
  -H "Content-Type: application/json" \
  -H "X-API-Key: test-key" \
  -d '{"content": "Hello from API!"}'

# Get list of all artifacts
curl -X GET http://localhost:5000/api/v1/artifacts \
  -H "X-API-Key: test-key"

# Get a specific artifact by ID
curl -X GET http://localhost:5000/api/v1/artifacts/sample_dataframe \
  -H "X-API-Key: test-key"

# Create a new artifact
curl -X POST http://localhost:5000/api/v1/artifacts \
  -H "Content-Type: application/json" \
  -H "X-API-Key: test-key" \
  -d '{"id": "api_artifact", "content": "console.log(\'Hello from API\');", "language": "javascript", "title": "API-Created Artifact", "type": "code"}'

# Update an existing artifact
curl -X PUT http://localhost:5000/api/v1/artifacts/api_artifact \
  -H "Content-Type: application/json" \
  -H "X-API-Key: test-key" \
  -d '{"content": "console.log(\'Updated from API\');"}'

# Start thinking process
curl -X POST http://localhost:5000/api/v1/thinking \
  -H "Content-Type: application/json" \
  -H "X-API-Key: test-key" \
  -d '{"action": "start"}'

# Add thinking step
curl -X POST http://localhost:5000/api/v1/thinking \
  -H "Content-Type: application/json" \
  -H "X-API-Key: test-key" \
  -d '{"action": "add_step", "title": "API Thinking Step", "body": "This step was added via the API interface."}'

# End thinking process
curl -X POST http://localhost:5000/api/v1/thinking \
  -H "Content-Type: application/json" \
  -H "X-API-Key: test-key" \
  -d '{"action": "end"}'
"""

print(curl_commands)

## Testing Structured Thinking Example

In [ ]:
# Demo of structured thinking
def demo_structured_thinking():
    chat.start_thinking()
    
    chat.add_thinking_step(
        "Data Analysis Started", 
        "Initializing analysis of the sales data for Q1 2023."
    )
    time.sleep(1.5)
    
    chat.add_thinking_step(
        "Examining Trends",
        """Looking at monthly performance trends:
        - January showed strong initial performance
        - February had a 7% decline in overall sales
        - March rebounded with 12% growth from February
        
        The quarterly trend is positive despite the February dip."""
    )
    time.sleep(2)
    
    chat.add_thinking_step(
        "Regional Comparison",
        """Comparing performance across regions:
        - North: 32% of total sales, 8% YoY growth
        - South: 28% of total sales, 6% YoY growth
        - East: 22% of total sales, 11% YoY growth
        - West: 18% of total sales, 9% YoY growth
        
        The East region showed the highest growth rate."""
    )
    time.sleep(2.5)
    
    chat.end_thinking()
    
    chat.send({"type": "chat_message", "content": "<h3>Q1 2023 Sales Analysis</h3><p>After analyzing the quarterly data, I can confirm that despite a February dip, overall performance was positive with the East region showing the strongest year-over-year growth at 11%.</p>"})

# Create a button to run the demo
from IPython.display import display, HTML
display(HTML('<button onclick="demo_structured_thinking()">Run Structured Thinking Demo</button>'))

# Define the function in the global scope so the button can call it
import IPython
IPython.get_ipython().user_ns['demo_structured_thinking'] = demo_structured_thinking